In [1]:
from mindspore import context
from mindspore.nn import Adam
from sponge import Sponge
from sponge import Molecule
from sponge import ForceField
from sponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

## 分子系统的复制与扩展

在分子对象的最基础的类Molecule中，支持了reduplicate和copy两种复制的模式，其实是同样的操作，不同点在于，reduplicate复制之后会默认把新生成的分子添加到此前定义的分子体系中去。而copy则是创建了一个新的对象，需要通过Molecule内置的append方法，才能把copy出来的对象添加到原始的分子系统中去。

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

## 优化器与积分器

在MindSPONGE与传统深度学习框架的架构图示意中，我们可以发现分子动力学模拟过程中的积分器与深度学习中的优化器，本质上其实是一回事，都需要获取到gradient然后再去对系统进行更新。而如果在分子动力学模拟的过程中直接使用传统优化器如GradientDescent和Adam等来进行能量最小化，也是一个很重要的过程。在执行动力学模拟过程之前，如果不对系统进行能量最小化，将很有可能在运行过程中出现梯度爆炸或者梯度消失的问题，会为系统的正常运行带来众多的不确定性。

![](../../docs/img.png)

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
md = Sponge(system, potential, opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [8]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b03.h5md', save_freq=10)

In [9]:
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-05-18 15:45:23
[MindSPONGE] Step: 0, E_pot: 242.99136
[MindSPONGE] Step: 10, E_pot: 163.51727
[MindSPONGE] Step: 20, E_pot: 93.06726
[MindSPONGE] Step: 30, E_pot: 15.81983
[MindSPONGE] Step: 40, E_pot: -66.221664
[MindSPONGE] Step: 50, E_pot: -134.5828
[MindSPONGE] Step: 60, E_pot: -182.6464
[MindSPONGE] Step: 70, E_pot: -220.4002
[MindSPONGE] Step: 80, E_pot: -252.21872
[MindSPONGE] Step: 90, E_pot: -278.47833
[MindSPONGE] Step: 100, E_pot: -302.31818
[MindSPONGE] Step: 110, E_pot: -328.98346
[MindSPONGE] Step: 120, E_pot: -354.57516
[MindSPONGE] Step: 130, E_pot: -373.94397
[MindSPONGE] Step: 140, E_pot: -392.10355
[MindSPONGE] Step: 150, E_pot: -411.2622
[MindSPONGE] Step: 160, E_pot: -430.96765
[MindSPONGE] Step: 170, E_pot: -449.49316
[MindSPONGE] Step: 180, E_pot: -464.83392
[MindSPONGE] Step: 190, E_pot: -476.66644
[MindSPONGE] Step: 200, E_pot: -485.10953
[MindSPONGE] Step: 210, E_pot: -491.31125
[MindSPONGE] Step: 220, E_pot: -496.3717
[

![](../../docs/tutorial_b03.gif)